In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import random as rd
import csv
import numpy as np
import math
from student_utils import *
from utils import *
import community
# from ipynb.fs.full.gen_inputs import add_weights2
from queue import PriorityQueue

# Building outputs from all algorithms

## Select the best output for each input

In [ ]:
def select_outputs(pickle_dir):
    pfiles = get_files_with_extension(pickle_dir, 'p')
    dic = {}
    meta_dic = {}
    for pf in pfiles:
        pobj = load_obj(pf)
        for item in pobj:
            # key=<./path/to/input_file>.in, value=(<./path/to/output_file.out>, cost)
            input_file = item[0]
            output_file = item[1]
            cost = item[2][0]
            if input_file in dic:
                dic[input_file].append((output_file, cost))
            else:
                dic[input_file] = [(output_file, cost)]
    if not os.path.isdir('final/outputs'):
        os.mkdir('final')
        os.mkdir('final/outputs')
    for lst in dic.values():
        best_output, best_cost = min(lst, key=lambda x: x[1])
        os.system('cp ' + best_output + ' final/outputs/')
        meta_dic[best_output] = best_cost
    save_obj(meta_dic, 'final/meta.p')
    write_data_to_file('final/meta.txt', meta_dic.items(), '\n')

In [ ]:
select_outputs('pickle_output')

# Metric TSP Solver

## Method 1: A polynomial-time Algo with cost <= 2*optimal

1. Take the MST of G.
2. Do a DPS on MST to get PT(pseudotour).
3. Go from PT to T* (Take each vertex only the first time it appears in PT)

In [ ]:
G1 = nx.complete_graph(10)
add_weights2(G1, 1, 10)
print(list(G1.edges().data()))
plt.subplot()
nx.draw(G1, node_color='yellow', with_labels=True)

In [ ]:
ta_homes = [0,1,2,3,4]
G1_sub = G1.subgraph(ta_homes)
print(list(G1_sub.edges().data()))
plt.subplot()
nx.draw(G1_sub, node_color='yellow', with_labels=True)

In [ ]:
PT = nx.minimum_spanning_tree(G1_sub)
# print(sorted(PT.edges(data=True)))
print(PT.edges(data=True))

plt.subplot()
nx.draw(PT, node_color='yellow', with_labels=True)

In [ ]:
# TODO: implement dfs yielding pseudotour
list(nx.dfs_edges(PT, source=0))

# Rao drives every TA home: this is a TSP

## Nearest Neighbor Algorithm

In [ ]:
def nn_solve(list_of_locations, list_of_homes, starting_car_location, adjacency_matrix, params=[]):
    start_idx = list_of_locations.index(starting_car_location)
    homes_idx = [list_of_locations.index(h) for h in list_of_homes]
    graph1, msg1 = adjacency_matrix_to_graph(adjacency_matrix)
    path = []
    drop_offs = {}
    current_idx = start_idx
    while homes_idx:
        lst_dist = [(h_idx, nx.shortest_path_length(graph1, current_idx, h_idx, 'weight')) for h_idx in homes_idx]
        idx, _ = min(lst_dist, key=lambda x: x[1])
        shortest_path = nx.shortest_path(graph1, current_idx, idx, 'weight')
        shortest_path.pop()
        path.extend(shortest_path)
        drop_offs[idx] = [idx]
        current_idx = idx
        homes_idx.remove(idx)
    last_shortest_path = nx.shortest_path(graph1, current_idx, start_idx, 'weight')
    path.extend(last_shortest_path)
    return path, drop_offs

In [ ]:
data = read_file('test_inputs/27_50.in')
num_loc, num_house, lst_loc, lst_house, start_loc, adj_mat = data_parser(data)
nn_solve(lst_loc, lst_house, start_loc, adj_mat)

## 2-Approximation using MST

- If the MST is connected then we can perform DFS and construct the path.
- If the MST is not connected then we need to connect all homes by finding shortest paths between them.
- To get the final path we look at each path in the list and check in the original graph, if there is an edge then we do nothing, but if it does not have an edge then we replace it with the shortest path between those nodes.

In [ ]:
def connect_homes(graph, nodes):
    newgraph = nx.Graph()
    newgraph.add_nodes_from(nodes)
    for i in range(len(nodes)):
        for j in range(i + 1, len(nodes)):
            w = nx.shortest_path_length(graph, nodes[i], nodes[j], 'weight')
            newgraph.add_edge(nodes[i], nodes[j], weight=w)
    return newgraph
    
def mst_solve(list_of_locations, list_of_homes, starting_car_location, adjacency_matrix, params=[]):
    start_idx = list_of_locations.index(starting_car_location)
    homes_idx = [list_of_locations.index(h) for h in list_of_homes]
    nodes = homes_idx + [start_idx]
    graph, msg = adjacency_matrix_to_graph(adjacency_matrix)
    newgraph = graph.subgraph(nodes)
    if not nx.is_connected(newgraph):
        newgraph = connect_homes(graph, nodes)
    mst = nx.minimum_spanning_tree(newgraph, 'weight')
    dfs = list(nx.dfs_edges(mst, start_idx))
    path = [start_idx]
    drop_offs = {}
    if start_idx in homes_idx:
        drop_offs[start_idx] = [start_idx]
    current_idx = -1
    while dfs:
        edge = dfs.pop(0)
        current_idx = edge[1]
        if current_idx not in path:
            path.append(current_idx)
            drop_offs[current_idx] = [current_idx]
    final_path = [path[0]]
    for i in range(len(path)-1):
        if graph.has_edge(path[i], path[i+1]):
            final_path.append(path[i+1])
        else:
            final_path.pop()
            final_path.extend(nx.shortest_path(graph, path[i], path[i+1], 'weight'))
    last_shortest_path = nx.shortest_path(graph, final_path[-1], start_idx, 'weight')
    final_path.pop()
    final_path.extend(last_shortest_path)
    return final_path, drop_offs

In [ ]:
# data = read_file('test_inputs/1_50.in')
data = read_file('inputs/79_200.in')
num_loc, num_house, lst_loc, lst_house, start_loc, adj_mat = data_parser(data)
mst_solve(lst_loc, lst_house, start_loc, adj_mat)

## 1.5-Approximation using Christofides Algorithm

In [ ]:
#code

In [ ]:
#test

## Branch-and-bound using the algorithm in the textbook

In [ ]:
NUM_DEC = 5  # number of decimal numbers for rounding

In [ ]:
#code

# list_of_locations, list_of_homes, starting_car_location, adjacency_matrix, 
def add_weights2(g, lower, upper):
    """Update the weight of each edge to be a random weight within the bounds."""
    for edge, data in g.edges.items():
        w = rd.uniform(lower, upper)
        data['weight'] = round(w, NUM_DEC)
        
def print_edge_weights(g, n=5):
    for edge, data in g.edges.items():
        if n <= 0:
            break
        n-=1
        print(edge, data)
        
def is_complete_graph(g):
    n = len(g.nodes())
    return (n * (n-1) / 2) == len(g.edges())

def convert_to_complete_graph(g, starting_car_location, list_of_homes, list_of_locations):
    rs = nx.floyd_warshall(g, weight='weight')
    result_g = nx.Graph()
    start_idx = list_of_locations.index(starting_car_location)
    homes_idx = [list_of_locations.index(h) for h in list_of_homes]
    subgraph_idx = [start_idx] + homes_idx
    subgraph = g.subgraph(subgraph_idx)
    subgraph = subgraph.copy()
    for i in range(len(subgraph_idx)):
        for j in range(i+1, len(subgraph_idx)):
            v0 = subgraph_idx[i]
            v1 = subgraph_idx[j]
            if not subgraph.has_edge(v0, v1):
                subgraph.add_edge(v0, v1, weight = rs[v0][v1])
#                 subgraph.edges[(i, j)]['weight'] = rs[i][j]
    return subgraph

def lowerbound1(g, V, S, a, b):
#     print("lowerbound1: S(current path) = ", S)
    V_set = set(V)
    S_set = set(S)
    V_minus_S = V_set.difference(S_set)
    
    cur_min_weight_a = float('inf')
    cur_min_weight_b = float('inf')
    
     # the lighest edge from node to V - S
    def find_min_weight(node):
        # COMMENT BELOW BECAUSE WE ARE DEALING WITH COMPLETE GRAPH node always has neighbors
#         node_nb_set = set(g.neighbors(node))
#         node_nb_intersect_V_minus_S = node_nb_set.intersection(V_minus_S)
#         cur_min_weight_node =  float('inf')
# #         print("for node", node, "its neighbors", node_nb_intersect_V_minus_S)
#         for node_nb_V_minus_S in node_nb_intersect_V_minus_S:
#             cur_min_weight_node = min(cur_min_weight_node,  g.edges[(node, node_nb_V_minus_S)]['weight'])
#         return cur_min_weight_node
    # *************************************************************
        cur_min_weight_node =  float('inf')
#         print("for node", node, "its neighbors", node_nb_intersect_V_minus_S)
        for nb in V_minus_S:
            cur_min_weight_node = min(cur_min_weight_node,  g.edges[(node, nb)]['weight'])
        return cur_min_weight_node

 
    # the lighest edge from a to V - S
    cur_min_weight_a = find_min_weight(a)
    
    # the lighest edge from b to V - S
    cur_min_weight_b = find_min_weight(b)
    
    # find the sum of weights of MST 
    V_minus_S_subgraph = g.subgraph(list(V_minus_S))
    
    mst = nx.minimum_spanning_tree(V_minus_S_subgraph)
    sum_weight_mst = sum([data['weight'] for edge, data in mst.edges.items()])
    
    assert cur_min_weight_a != float('inf') and cur_min_weight_b != float('inf')
    return cur_min_weight_a + cur_min_weight_b + sum_weight_mst

def branch_and_bound(g, starting_vertex, lower_bound):
#     active_subproblems = PriorityQueue()
    active_subproblems = []
    active_subproblems.append((starting_vertex, [starting_vertex], starting_vertex))
#     active_subproblems.put((lower_bound(g, g.nodes(), [starting_vertex], starting_vertex, starting_vertex), (starting_vertex, [starting_vertex], starting_vertex)))
    best_so_far = float('inf')
    best_path = []
#     while active_subproblems.qsize() > 0:
    while len(active_subproblems) > 0:
        # choose
#         active_sub = active_subproblems.get()[1]
        active_sub = active_subproblems.pop()
#         print("active_sub=", active_sub)
        # expand
        V_set = set(g.nodes())
        S_set = set(tuple(active_sub[1]))
        V_minus_S = V_set.difference(S_set)
        
        for x in V_minus_S:
            path = active_sub[1]+[x]
            P = (active_sub[0], path , x)
#             print("P = ", P)
            # complete sol
            total_w = 0
            
            
            if len(path) == len(g.nodes()):
                for i in range(len(path)-1):
#                     print("***********")
#                     print(path[i], path[i+1])
                    total_w += g.edges[(path[i], path[i+1])]['weight']
#                 print("considering ", total_w, path)
                if total_w < best_so_far:
                    best_so_far = total_w
                    best_path = path
        
           
            # if lowerbound < bestsofar add Pi to S 
            else:
                cur_lower_bound = lower_bound(g, g.nodes(), path, active_sub[0], x)
                if cur_lower_bound < best_so_far:
#                     active_subproblems.put((cur_lower_bound, P))
                    active_subproblems.append(P)
                    
                #(g, V, S, a, b): 
    return best_so_far, best_path

In [ ]:
#test
G1 = nx.complete_graph(11)
add_weights2(G1, 1, 100000000)
nx.draw(G1, node_color='yellow', with_labels=True)

print_edge_weights(G1, len(G1.edges()))

# converted_g=convert_to_complete_graph(G1, 0, [1, 2, 3], [0,1,2,3])  
# print(is_complete_graph(converted_g))        
# nx.draw(converted_g, node_color='yellow', with_labels=True)

# print(lowerbound(G1, G1.nodes(), [0], 0, 0))
# branch_and_bound(G1, 0, lowerbound1)
# q = PriorityQueue()
# q.put((1, "a"))
# q.put((-1, "b"))
# # print(q.get())
# # print(q.get())
# print(q.qsize())
# print(set(tuple([1,2,3])))

In [ ]:
branch_and_bound(G1, 0, lowerbound1)

In [ ]:
from itertools import permutations

def get_cost_of_path(g, path):
    total = 0
    for i in range(len(path)-1):
        total += g.edges[(path[i], path[i+1])]['weight']
    return total
        
perm = permutations(G1.nodes())
all_cost = []
# print(list(perm))
cur_min = float("inf")
cur_min_path = []
for p in perm:
    p = list(p)
#     p.append(0)
    if p[0] == 0:
        cur_cost = get_cost_of_path(G1, p)
        all_cost.append(cur_cost)
#         print(cur_cost, p)

        if cur_cost < cur_min:
            cur_min = cur_cost
            cur_min_path = p
print(cur_min)
print(cur_min_path)
# print(all_cost)

## Branch-and-bound using another lower-bound

In [ ]:
#code

In [ ]:
#test

# Rao drives TAs to some dropoff locations (clustering)

In [ ]:
data = read_file('inputs/79_200.in')
num_loc, num_house, list_of_locations, list_of_homes, starting_car_location, adjacency_matrix = data_parser(data)
graph, msg = adjacency_matrix_to_graph(adjacency_matrix)
start_idx = list_of_locations.index(starting_car_location)
homes_idx = [list_of_locations.index(h) for h in list_of_homes]

plt.figure(1)
nx.draw(graph, with_labels=True)

#first compute the best partition
partition = community.best_partition(graph)

#drawing
plt.figure(2)
size = float(len(set(partition.values())))
pos = nx.spring_layout(graph)
count = 0.
for com in set(partition.values()) :
#     print(type(com))
    count = count + 1.
    list_nodes = [nodes for nodes in partition.keys()
                                if partition[nodes] == com]
    nx.draw_networkx_nodes(graph, pos, list_nodes, node_size = 20,
                                node_color = str(count / size))
print('Number of communities: ', count)

nx.draw_networkx_edges(graph, pos, alpha=0.5)
plt.show()

In [ ]:
def count_homes(k, graph, homes_idx):
    lst = []
    for node in graph.nodes():
        lst_homes = []
        neighbors = list(graph.neighbors(node))
        for n in neighbors:
            if n in homes_idx:
                lst_homes.append(n)
        if k == 2:
            for n in neighbors:
                for nn in graph.neighbors(n):
                    if nn in homes_idx:
                        lst_homes.append(nn)
        if len(lst_homes) > 0:
            lst.append((node, lst_homes))
    return lst

def init_pq(lst, graph, cur_idx):
    shortest_paths = nx.shortest_path_length(graph, cur_idx, weight='weight')
    pq = PriorityQueue()
    for i in lst:
        node = i[0]
        pq.put((shortest_paths[node], i))
    return pq
    
def k_layers_cluster(k, list_of_locations, list_of_homes, starting_car_location, adjacency_matrix):
    start_idx = list_of_locations.index(starting_car_location)
    homes_idx = [list_of_locations.index(h) for h in list_of_homes]
    graph, msg = adjacency_matrix_to_graph(adjacency_matrix)
    path = []
    drop_offs = {}
    lst = count_homes(k, graph, homes_idx)
    cur_idx = start_idx
    while homes_idx:
        pq = init_pq(lst, graph, cur_idx)
        cost, center_homes = pq.get()
        center = center_homes[0]
        lst_homes = center_homes[1]
        lst_copy = lst_homes.copy()
        for h in lst_copy:
            if h in homes_idx:
                homes_idx.remove(h)
            else:
                lst_homes.remove(h)
        if lst_homes:
            shortest_path = nx.shortest_path(graph, cur_idx, center, 'weight')
            shortest_path.pop()
            path.extend(shortest_path)
            drop_offs[center] = lst_homes
            cur_idx = center
        lst.remove(center_homes)
    last_shortest_path = nx.shortest_path(graph, cur_idx, start_idx, 'weight')
    path.extend(last_shortest_path)
    return path, drop_offs
    
def nn_cluster_solve(list_of_locations, list_of_homes, starting_car_location, adjacency_matrix, params=[]):
    return k_layers_cluster(2, list_of_locations, list_of_homes, starting_car_location, adjacency_matrix)

In [ ]:
data = read_file('inputs/79_200.in')
num_loc, num_house, list_of_locations, list_of_homes, starting_car_location, adjacency_matrix = data_parser(data)
nn_cluster_solve(list_of_locations, list_of_homes, starting_car_location, adjacency_matrix)

# Greedy Clustering

In [ ]:
def greedy_clustering(list_of_locations, list_of_homes, starting_car_location, adjacency_matrix, params=[]):
    # Count adjacent homes for each vertice and add them to the PQ
    start_idx = list_of_locations.index(starting_car_location)
    homes_idx = [list_of_locations.index(h) for h in list_of_homes]
    g, msg1 = adjacency_matrix_to_graph(adjacency_matrix)
    PQ = PriorityQueue()
    car_path = []
    drop_off_locations = {}
    
    # initilize PQ
    for v in g.nodes():
        homes = []
        for nb in g.neighbors(v):
            if nb in homes_idx:
                homes.append(nb)
        PQ.put((len(homes), (v, homes)))
        
#     (prev_dropoff_loc, dropoff_homes) = PQ.get()[1]
#     dropoff_copy = dropoff_homes.copy()
#     for h in dropoff_copy:
#         if h in homes_idx:
#             homes_idx.remove(h)
#         else:
#             dropoff_homes.remove(h)
#     if dropoff_homes:
#         car_path.extend(nx.shortest_path(g, start_idx, prev_dropoff_loc, weight='weight'))
#         drop_off_locations[prev_dropoff_loc] = dropoff_homes
    
    
    cur_idx = start_idx
    while homes_idx:
        (dropoff_loc, dropoff_homes) = PQ.get()[1]
        dropoff_copy = dropoff_homes.copy()
        for h in dropoff_copy:
            if h in homes_idx:
                homes_idx.remove(h)
            else:
                dropoff_homes.remove(h)
        if dropoff_homes:
            s_path = nx.shortest_path(g, cur_idx, dropoff_loc, weight='weight')
            cur_idx = dropoff_loc
            s_path.pop()
            car_path.extend(s_path)
            drop_off_locations[dropoff_loc] = dropoff_homes
    
    car_path.extend(nx.shortest_path(g, cur_idx, start_idx, weight='weight'))
    return car_path, drop_off_locations   

In [ ]:
data = read_file('test_inputs/27_200.in')
num_loc, num_house, lst_loc, lst_house, start_loc, adj_mat = data_parser(data)
car_path, drop_off_locations = greedy_clustering(lst_loc, lst_house, start_loc, adj_mat)

g, msg1 = adjacency_matrix_to_graph(adj_mat)
print(car_path)
print(cost_of_solution(g, car_path, drop_off_locations))

In [ ]:
data = read_file('test_inputs/116_50.in')
num_loc, num_house, lst_loc, lst_house, start_loc, adj_mat = data_parser(data)
car_path, drop_off_locations = greedy_clustering(lst_loc, lst_house, start_loc, adj_mat)

g, msg1 = adjacency_matrix_to_graph(adj_mat)
print(car_path)
print(cost_of_solution(g, car_path, drop_off_locations))